由原始mat格式的标注，转换成QPIC提供的HOIA标注格式

In [1]:
from scipy.io import loadmat
from pprint import pprint
import json

In [2]:
raw_ann = loadmat('../../annotations/raw/anno_bbox.mat')
raw_ann.keys()

dict_keys(['__header__', '__version__', '__globals__', 'bbox_train', 'bbox_test', 'list_action'])

In [3]:
raw_train, raw_test = raw_ann['bbox_train'][0], raw_ann['bbox_test'][0]

In [4]:
label_info = json.load(open('../../configs/label_info.json'))
hoi_info = label_info['hoi_info']

In [5]:
def mat2json(mat_anns):
    new_anns, emptys, duplicate = [], [], set()
    for i, (filename, size, anns) in enumerate(mat_anns):
        filename = filename.item()
        img_id = int(filename[-12:-4])

        W, H, C = size.item()
        W, H = W.item(), H.item()

        ins_anns, rel_anns, ins_map, rel_set = [], [], {}, set()
        for hoi, sub_bboxes, obj_bboxes, pairs, invis in anns[0]:
            hoi = hoi.item()
            rel = hoi_info[str(hoi)]['ann_rel_id']
            obj = hoi_info[str(hoi)]['ann_obj_id']

            if len(pairs) < 1:
                assert invis.item() == 1
                continue

            # 每个包含若干种hoi
            # 一种hoi涉及多对sub-obj
            sub_ids, obj_ids = [], []  # 某种hoi涉及的sub、obj
            for bbox in sub_bboxes[0]:
                x1, x2, y1, y2 = (i.item() for i in bbox.item())
                bbox = (x1, y1, x2, y2)
                assert x2 <= W and y2 <= H, f'{i}, {filename}, {bbox}, {W}, {H}'
                key = (bbox, 1)  # 1 是 human label id
                if key not in ins_map:
                    ins_map[key] = len(ins_anns)
                    ins_anns.append(dict(bbox=bbox, category_id=1))
                sub_ids.append(ins_map[key])
            for bbox in obj_bboxes[0]:
                x1, x2, y1, y2 = (i.item() for i in bbox.item())
                bbox = (x1, y1, x2, y2)
                assert x2 <= W and y2 <= H, f'{i}, {filename}, {bbox}, {W}, {H}'
                key = (bbox, obj)
                if key not in ins_map:
                    ins_map[key] = len(ins_anns)
                    ins_anns.append(dict(bbox=bbox, category_id=obj))
                obj_ids.append(ins_map[key])

            for sub_idx, obj_idx in pairs:
                sub_id, obj_id = sub_ids[sub_idx-1], obj_ids[obj_idx-1]
                assert sub_id != obj_id, f'{i}, {filename}, {sub_idx}, {obj_idx}'
                if (sub_id, obj_id, hoi) not in rel_set:
                    rel_set.add((sub_id, obj_id, hoi))
                    rel_anns.append(dict(
                        subject_id=sub_id,
                        object_id=obj_id,
                        category_id=rel,
                        hoi_category_id=hoi
                    ))
                else:
                    duplicate.add(i)

        if len(rel_anns) == 0:
            emptys.append((i, filename))
            continue

        new_anns.append(dict(
            file_name=filename,
            img_id=img_id,
            width=W,
            height=H,
            annotations=ins_anns,
            hoi_annotations=rel_anns
        ))
    print('total:', len(mat_anns))
    print('empty:', len(emptys))
    print('valid:', len(new_anns))
    print('with duplicate:', len(duplicate))
    return new_anns, emptys, list(duplicate)

In [6]:
print('train')
new_train, train_empty, train_duplicate = mat2json(raw_train)
print('\ntest')
new_test, test_empty, test_duplicate = mat2json(raw_test)

train
total: 38118
empty: 485
valid: 37633
with duplicate: 1

test
total: 9658
empty: 112
valid: 9546
with duplicate: 0


In [77]:
pprint(new_train[0], width=100, compact=True, sort_dicts=False)  # 1

{'file_name': 'HICO_train2015_00000001.jpg',
 'img_id': 1,
 'width': 640,
 'height': 480,
 'annotations': [{'bbox': (208, 33, 427, 300), 'category_id': 1},
                 {'bbox': (59, 98, 572, 405), 'category_id': 4},
                 {'bbox': (213, 20, 438, 357), 'category_id': 1},
                 {'bbox': (77, 115, 583, 396), 'category_id': 4},
                 {'bbox': (206, 33, 427, 306), 'category_id': 1},
                 {'bbox': (61, 100, 571, 401), 'category_id': 4},
                 {'bbox': (209, 26, 444, 317), 'category_id': 1},
                 {'bbox': (59, 99, 579, 395), 'category_id': 4}],
 'hoi_annotations': [{'subject_id': 0, 'object_id': 1, 'category_id': 73, 'hoi_category_id': 153},
                     {'subject_id': 2, 'object_id': 3, 'category_id': 77, 'hoi_category_id': 154},
                     {'subject_id': 4, 'object_id': 5, 'category_id': 88, 'hoi_category_id': 155},
                     {'subject_id': 6, 'object_id': 7, 'category_id': 99, 'hoi_categor

In [78]:
pprint(new_test[0], width=100, compact=True, sort_dicts=False)  # 1

{'file_name': 'HICO_test2015_00000001.jpg',
 'img_id': 1,
 'width': 640,
 'height': 427,
 'annotations': [{'bbox': (320, 306, 359, 349), 'category_id': 1},
                 {'bbox': (270, 303, 311, 350), 'category_id': 1},
                 {'bbox': (148, 345, 376, 414), 'category_id': 15}],
 'hoi_annotations': [{'subject_id': 0, 'object_id': 2, 'category_id': 88, 'hoi_category_id': 246},
                     {'subject_id': 1, 'object_id': 2, 'category_id': 88, 'hoi_category_id': 246}]}


In [79]:
json.dump(new_train, open('train.json', 'w'))
json.dump(new_test, open('test.json', 'w'))